In [94]:
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout,Flatten, BatchNormalization
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K
# load ascii text and covert to lowercase

In [27]:
import pickle
import numpy as np
import keras

In [3]:
from keras.utils import to_categorical

data = pickle.load( open( "jazz123.txt", "rb" ))
lengths = [len(x) for x in data]
idx = np.cumsum(lengths)
d = sum(data,[])
e = to_categorical(d)
data_rec = [e[:idx[0]]] + [e[idx[t]:idx[t+1]] for t in range(len(idx) - 2)]

In [72]:
data_rec[1][0]

128

Version #1: 49 -> 1 

In [6]:
dataX = []
dataY = []

seq_length = 50

for seq in data_rec: 
    n_chars = len(seq)
    for i in range(0, n_chars - seq_length, 1):
        if (seq[i][0] == 1.0 or seq[i][1] == 1.0 ):
            continue
        seq_in = seq[i:i + seq_length]
        seq_out = seq[i + seq_length]
        dataX.append(seq_in)
        dataY.append(seq_out)
n_patterns = len(dataX)


In [7]:
len(dataX)

1362200

Version #2: 50 целиком

In [23]:
len(dataX)

1362200

In [29]:
number = 100000

X = numpy.reshape(dataX[:number], (number, seq_length, 128))
y = np.array(dataY[:number])

In [86]:
number = 10000
for i in range(120):
    
    X = numpy.reshape(dataX[i * number:(i+1) * number], (number, seq_length, 128))
    y = np.array(dataY[i * number:(i+1) * number])
    
    with open("data_X_" + str(i), "wb") as fp:  
        np.save(fp, X, allow_pickle = True, fix_imports=False)
    
    with open("data_y_" + str(i), "wb") as fp:  
        np.save(fp, y, allow_pickle = True, fix_imports=False)
    

KeyboardInterrupt: 

In [30]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
filepath

'weights-improvement-{epoch:02d}-{loss:.4f}.hdf5'

In [98]:
model = Sequential()
model.add(LSTM(250, input_shape=(X.shape[1], X.shape[2]), return_sequences = True, return_state = False))
model.add(BatchNormalization())
model.add(LSTM(225, return_sequences = True, return_state = False))
model.add(BatchNormalization())
model.add(LSTM(200, return_sequences = True, return_state = False))
model.add(BatchNormalization())
model.add(LSTM(175, return_sequences = True, return_state = False))
model.add(BatchNormalization())
model.add(LSTM(150, return_sequences = True, return_state = False))
model.add(BatchNormalization())
model.add(LSTM(128, return_sequences = False, return_state = False))#added

#model.add(Dropout(0.2))

#model.add(Flatten())
model.add(Dense(y.shape[1], activation='softmax'))

filepath="2Xmodel-{epoch:02d}-{loss:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only = True, mode='min')
callbacks_list = [checkpoint]

opt = keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy', optimizer=opt)


In [96]:
number = len(dataX[0][0])
X = numpy.reshape(dataX[:number], (number, seq_length, 128))
y = np.array(dataY[:number])
model.fit(X, y, epochs = 1, verbose=1, batch_size = 128, callbacks = callbacks_list, shuffle = True)

Epoch 1/1
128/128 [==============================] - 4s 30ms/step - loss: 4.8941

Epoch 00001: loss improved from inf to 4.89408, saving model to 2Xmodel-01-4.894.hdf5


In [7]:
model.load_weights("model-01-1.302.hdf5")

In [53]:
[1,2,3][0,1]

TypeError: list indices must be integers or slices, not tuple

In [162]:
n_epochs = 100
number = 100000


for epoch in range(n_epochs):
    
    print("downloading data....")
    rando = np.random.choice(range(len(dataX)), number, replace = False)
    
    X = numpy.reshape([dataX[i] for i in rando], (number, seq_length, 128))
    y = np.array([dataY[i] for i in rando])
    
    print("epoch: %d"%(epoch))
    
    model.fit(X, y, epochs = 1, verbose=1, batch_size = 128, callbacks = callbacks_list, shuffle = True)
    #np.random.shuffle(L)
    
    
    #for j, i in enumerate(L):
        
        
    #    print("downloading data....")
        
    #    X = numpy.reshape(dataX[i * number:(i+1) * number], (number, seq_length, 128))
    #    y = np.array(dataY[i * number:(i+1) * number])
        
        
        #print("epoch: %d, part: %d, number: %d" % (epoch, j, i))
        #model.fit(X, y, epochs = 1, verbose=1, batch_size = 128, callbacks = callbacks_list, shuffle = True)

downloading data....
epoch: 0
Epoch 1/1
100000/100000 [==============================] - 239s 2ms/step - loss: 0.8717

Epoch 00001: loss did not improve from 0.85919
downloading data....
epoch: 1
Epoch 1/1
100000/100000 [==============================] - 239s 2ms/step - loss: 0.8557

Epoch 00001: loss improved from 0.85919 to 0.85567, saving model to 2Xmodel-01-0.856.hdf5
downloading data....
epoch: 2
Epoch 1/1
100000/100000 [==============================] - 240s 2ms/step - loss: 0.8582

Epoch 00001: loss did not improve from 0.85567
downloading data....
epoch: 3
Epoch 1/1
100000/100000 [==============================] - 241s 2ms/step - loss: 0.8572

Epoch 00001: loss did not improve from 0.85567
downloading data....
epoch: 4
Epoch 1/1
100000/100000 [==============================] - 240s 2ms/step - loss: 0.8559

Epoch 00001: loss did not improve from 0.85567
downloading data....
epoch: 5
Epoch 1/1
100000/100000 [==============================] - 239s 2ms/step - loss: 0.8588

Epoch 00

100000/100000 [==============================] - 235s 2ms/step - loss: 0.8313

Epoch 00001: loss did not improve from 0.82983
downloading data....
epoch: 47
Epoch 1/1
100000/100000 [==============================] - 236s 2ms/step - loss: 0.8493

Epoch 00001: loss did not improve from 0.82983
downloading data....
epoch: 48
Epoch 1/1
100000/100000 [==============================] - 240s 2ms/step - loss: 0.8383

Epoch 00001: loss did not improve from 0.82983
downloading data....
epoch: 49
Epoch 1/1
100000/100000 [==============================] - 239s 2ms/step - loss: 0.8343

Epoch 00001: loss did not improve from 0.82983
downloading data....
epoch: 50
Epoch 1/1
100000/100000 [==============================] - 238s 2ms/step - loss: 0.8368

Epoch 00001: loss did not improve from 0.82983
downloading data....
epoch: 51
Epoch 1/1
100000/100000 [==============================] - 238s 2ms/step - loss: 0.8361

Epoch 00001: loss did not improve from 0.82983
downloading data....
epoch: 52
Epoch 1/

KeyboardInterrupt: 

In [153]:
np.max(prediction, axis = 1)

array([0.46667632], dtype=float32)

In [157]:
np.array(prediction[0][sorted(range(128), key = lambda x:prediction[0,x], reverse = True)] / np.max(prediction, axis = 1) * 100, "int32")

array([100,  42,   7,   5,   2,   2,   1,   1,   1,   1,   1,   1,   1,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])

In [40]:
rando = np.random.choice(range(1300000), 100000, replace=False)
print(len(set(rando)))

100000


In [86]:
model.save("total_sunday.hdf5")

In [87]:
"%d + %d"%(1,2)

'1 + 2'

In [98]:
model.fit(X, y, epochs = 2, verbose=1, batch_size = 256)

Epoch 1/2
100000/100000 [==============================] - 261s 3ms/step - loss: 1.7632
Epoch 2/2
 71168/100000 [====================>.........] - ETA: 1:15 - loss: 1.6806

KeyboardInterrupt: 

In [31]:
X_in = X[0].reshape((1, 50, 128))

In [35]:
model.predict(X_in)

array([[3.02089757e-04, 7.29699731e-01, 4.81484392e-07, 5.07727975e-07,
        4.65268670e-07, 4.97117639e-07, 5.17919318e-07, 4.38293711e-07,
        4.94290703e-07, 5.40053734e-05, 4.33493227e-07, 4.24668684e-07,
        4.55670772e-07, 5.74965270e-07, 4.84270458e-07, 4.57068722e-07,
        5.48520063e-07, 4.43684343e-07, 4.98198290e-07, 4.62957814e-07,
        4.50848489e-07, 1.07666267e-06, 5.57208296e-06, 2.07643416e-06,
        6.50653919e-06, 6.11844598e-06, 1.63248351e-05, 2.08976446e-04,
        6.75121919e-05, 1.13725034e-03, 2.89812335e-04, 2.44551688e-03,
        9.87526611e-04, 7.16817472e-03, 5.92686329e-03, 5.87560795e-03,
        5.66303264e-03, 3.21254390e-03, 3.52350287e-02, 5.68942493e-03,
        7.39225559e-03, 3.86381447e-02, 1.42651238e-03, 7.06781447e-02,
        6.86901622e-03, 3.45782414e-02, 1.13773420e-02, 2.76215211e-03,
        1.95123709e-03, 2.37547420e-03, 6.32626330e-03, 6.03665481e-04,
        1.73663010e-03, 8.64834583e-04, 9.80679106e-05, 7.728250

In [85]:
np.random.choice([1,2,3,4,5,6,7,8,9,10])

1

In [20]:
def take_one(pred):
    a = np.log(pred[0]) / 1.0 
    dist = np.exp(a)/np.sum(np.exp(a)) 
    choices = range(len(a))
    return np.random.choice(choices, p=dist)

In [196]:
ch = range(1300000)
ch = 1300
X_in = np.reshape(dataX[ch] ,(1, 50, 128))
#X_in = np.reshape(dataX[np.random.choice(ch)] ,(1, 50, 128))
#X_in = np.reshape([to_categorical(np.random.choice(range(128)), num_classes = 128) if i % 2 == 0 else to_categorical(1, num_classes = 128) for i in range(50)],(1, 50, 128))
#X_in = np.reshape([to_categorical(i, num_classes = 128) if i % 2 == 0 else to_categorical(1, num_classes = 128) for i in range(34,84)],(1, 50, 128))
new_track = [np.argmax(x) for x in X_in[0]] + [120, 1, 1, 1]
p = 2
previous = 1
for i in range(1200):
    prediction = model.predict(X_in)
    print(np.array(prediction[0][sorted(range(128), key = lambda x:prediction[0,x], reverse = True)] / np.max(prediction, axis = 1) * 100, "int32")[:10])
    '''
    if p!= 1:
        prediction[0,p] /= 10.0 
    else:
        prediction[0,1] *= 2.0
    '''
    prediction[0,previous] = 0.0
    p = take_one(prediction)
    if (p != 1 and p != 0):
        previous = p
    #print(p)
    #previous_note = p
    new_track.append(p)
    X_in = list(X_in[0])[1:]
    p_cat = to_categorical(p, num_classes = 128)
    
    X_in.append(p_cat)
    
    X_in = np.reshape(X_in,(1, 50, 128))
    

[100  57  11   9   7   7   3   1   1   1]
[100  19   7   3   3   2   1   1   0   0]
[100  92  40  38  11   6   3   3   2   1]


C:\Users\Charles\Anaconda3\envs\condathree\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


[100  17   7   7   2   1   1   0   0   0]
[100  43  22  20  20   5   5   3   3   1]
[100  17  14  12  10   5   3   1   0   0]
[100  11   8   4   3   1   0   0   0   0]
[100  58  38  20  14  11   5   1   0   0]
[100   2   0   0   0   0   0   0   0   0]
[100  71  46  45  17   5   4   3   2   1]
[100   1   1   0   0   0   0   0   0   0]
[100  36  20  17  13   5   4   3   2   1]
[100  53  25  25  10   5   4   2   2   2]
[100  24  15   5   4   3   2   2   2   1]
[100  27   6   4   3   2   2   1   1   1]
[100  26  12   7   5   3   3   2   2   2]
[100  10   5   5   3   2   2   2   1   1]
[100   4   4   2   2   1   1   1   0   0]
[100   4   3   2   1   1   1   1   0   0]
[100   8   4   3   2   2   2   1   1   1]
[100  26  13   7   4   4   4   3   3   2]
[100  81  44  29  18  18  14  10   8   7]
[100   9   3   1   0   0   0   0   0   0]
[100  58   5   3   3   2   2   2   1   1]
[100  96   5   4   3   3   2   2   2   2]
[100  14   0   0   0   0   0   0   0   0]
[100  25   2   1   0   0   0   0  

[100   6   5   4   3   2   2   1   1   1]
[100  30  22  18  13  11   8   6   5   2]
[100   3   1   1   1   0   0   0   0   0]
[100  41  27  15  10   9   8   7   7   6]
[100   6   3   2   2   1   1   1   0   0]
[100  21  11   6   4   3   1   1   1   1]
[100   5   3   1   1   1   1   0   0   0]
[100  12   9   4   3   3   3   2   2   2]
[100   0   0   0   0   0   0   0   0   0]
[100  12   4   3   1   0   0   0   0   0]
[100  42  25  19  17   4   2   1   1   1]
[100   5   2   2   1   1   0   0   0   0]
[100  21  19  15   6   3   2   2   2   2]
[100   7   5   4   4   3   3   1   1   1]
[100   3   3   2   2   1   1   1   1   1]
[100  21   5   4   4   4   4   3   2   2]
[100  16   5   4   3   2   2   2   2   2]
[100  24  21  17  14  13  12  11   9   9]
[100  19  10   8   5   5   5   4   4   4]
[100  14  13  12   8   7   5   4   4   3]
[100  10   5   5   5   1   0   0   0   0]
[100   4   2   2   2   2   1   1   1   1]
[100   5   3   1   1   0   0   0   0   0]
[100  14   9   8   7   6   4   4  

[100   0   0   0   0   0   0   0   0   0]
[100  83   0   0   0   0   0   0   0   0]
[100   0   0   0   0   0   0   0   0   0]
[100   0   0   0   0   0   0   0   0   0]
[100   0   0   0   0   0   0   0   0   0]
[100   0   0   0   0   0   0   0   0   0]
[100   0   0   0   0   0   0   0   0   0]
[100   1   0   0   0   0   0   0   0   0]
[100   4   0   0   0   0   0   0   0   0]
[100   2   0   0   0   0   0   0   0   0]
[100   2   1   0   0   0   0   0   0   0]
[100  84  54  46  40  38  34  31  26  13]
[100   0   0   0   0   0   0   0   0   0]
[100  19   3   1   0   0   0   0   0   0]
[100  99  33  12  10   8   5   2   1   1]
[100  13   4   1   1   1   0   0   0   0]
[100  49  14   6   4   3   3   0   0   0]
[100  38  14   1   1   1   0   0   0   0]
[100  68  35  28   7   4   2   1   1   0]
[100  43   5   2   2   2   1   1   0   0]
[100  41   4   4   4   3   2   2   1   1]
[100  13   2   1   0   0   0   0   0   0]
[100  34  31  13   2   2   2   2   2   1]
[100  46  14   6   3   1   1   1  

[100   0   0   0   0   0   0   0   0   0]
[100   1   0   0   0   0   0   0   0   0]
[100   1   0   0   0   0   0   0   0   0]
[100   1   0   0   0   0   0   0   0   0]
[100   3   0   0   0   0   0   0   0   0]
[100   1   0   0   0   0   0   0   0   0]
[100   9   2   0   0   0   0   0   0   0]
[100  40   3   0   0   0   0   0   0   0]
[100  66  47  43  42  15  13   9   8   6]
[100   0   0   0   0   0   0   0   0   0]
[100   3   0   0   0   0   0   0   0   0]
[100   7   0   0   0   0   0   0   0   0]
[100   0   0   0   0   0   0   0   0   0]
[100   1   0   0   0   0   0   0   0   0]
[100   0   0   0   0   0   0   0   0   0]
[100   8   0   0   0   0   0   0   0   0]
[100  57   0   0   0   0   0   0   0   0]
[100  68  54  47  33  20   7   4   4   3]
[100   0   0   0   0   0   0   0   0   0]
[100   0   0   0   0   0   0   0   0   0]
[100   0   0   0   0   0   0   0   0   0]
[100   1   0   0   0   0   0   0   0   0]
[100   2   0   0   0   0   0   0   0   0]
[100   2   0   0   0   0   0   0  

[100   9   6   4   0   0   0   0   0   0]
[100  27  24  17  15  10   6   3   1   0]
[100  13   5   1   0   0   0   0   0   0]
[100  22   0   0   0   0   0   0   0   0]
[100  12   3   1   1   0   0   0   0   0]
[100  12   9   2   1   0   0   0   0   0]
[100   9   3   1   0   0   0   0   0   0]
[100  10   3   0   0   0   0   0   0   0]
[100   7   3   0   0   0   0   0   0   0]
[100   6   4   1   0   0   0   0   0   0]
[100   8   5   1   0   0   0   0   0   0]
[100  17   6   1   1   0   0   0   0   0]
[100  50  23   9   6   5   3   2   2   2]
[100  49  15  12  10   7   5   3   2   1]
[100  62  45  38  33  31  23  20  14  14]
[100   0   0   0   0   0   0   0   0   0]
[100  46   0   0   0   0   0   0   0   0]
[100   1   0   0   0   0   0   0   0   0]
[100   0   0   0   0   0   0   0   0   0]
[100   0   0   0   0   0   0   0   0   0]
[100   0   0   0   0   0   0   0   0   0]
[100   0   0   0   0   0   0   0   0   0]
[100   0   0   0   0   0   0   0   0   0]
[100   0   0   0   0   0   0   0  

[100  11   7   6   4   4   4   4   3   3]
[100   4   3   2   2   1   1   1   1   1]
[100  25  10   9   7   6   6   4   4   3]
[100   4   3   3   3   3   2   1   1   1]
[100  16  16   8   3   2   2   1   1   1]
[100  25  23  18  15  10   9   6   6   6]
[100  99  85  73  41  38  30  22  15  12]
[100   2   1   1   1   1   1   0   0   0]
[100  39   3   2   1   0   0   0   0   0]
[100  12  11   7   6   6   4   4   3   2]
[100   9   7   7   7   5   4   3   3   2]
[100   6   5   3   3   3   2   2   2   2]
[100  18   9   6   5   5   4   3   3   2]
[100   9   7   5   4   3   3   2   2   1]
[100  16   2   2   2   2   2   2   1   1]
[100  15  15  15  14  11  10   8   7   5]
[100   6   5   4   3   2   2   2   1   1]
[100  18  17  11   8   4   3   3   2   2]
[100  12  12   8   4   3   2   2   1   1]
[100  22  20   7   4   3   2   2   2   2]
[100   9   4   3   3   2   2   2   1   1]
[100  12   7   6   5   3   2   2   2   2]
[100  46  33  23  22  20  16  15   9   8]
[100   5   4   2   2   1   1   1  

[100  12   1   0   0   0   0   0   0   0]
[100  31  27  22  22  18  18  10   6   5]
[100   0   0   0   0   0   0   0   0   0]
[100  44   0   0   0   0   0   0   0   0]
[100  57   3   3   2   1   1   0   0   0]
[100  45   1   0   0   0   0   0   0   0]
[100  61  47  43  28  18  14  11   9   9]
[100   0   0   0   0   0   0   0   0   0]
[100  18   0   0   0   0   0   0   0   0]
[100  22   1   1   1   0   0   0   0   0]
[100  96   3   2   2   1   1   0   0   0]
[100   8   6   1   1   1   0   0   0   0]
[100   9   4   2   2   1   0   0   0   0]


In [22]:
len(X)

128

In [197]:
create_track_new(new_track)

0


In [184]:
from midiutil import MIDIFile

def create_track_new(seq):
    
    grain = 16
    one_tact = 16
    tacts_per_min = 75
    tempo    = one_tact * tacts_per_min  # In BPM
    volume   = 100  # 0-127, as per the MIDI standard
    track    = 0
    channel  = 0
    time     = 0# to learn
    
    MyMIDI = MIDIFile(1)
    MyMIDI.addTempo(track, time, tempo)
    
    
    i = 0
    tick_count = 0
    while(i < len(seq)):
        
        if i == len(seq) - 1:
            MyMIDI.addNote(track, channel, seq[i], tick_count, one_tact / grain, volume)
            tick_count = tick_count + one_tact / grain
            i = i + 1
        else:
            if seq[i + 1] != 1:
                MyMIDI.addNote(track, channel, seq[i], tick_count, one_tact / grain, volume)
                tick_count = tick_count + one_tact / grain 
                i = i + 1
            else:
                i_prev = i
                i = i + 1
                while(seq[i] == 1 and i < len(seq) - 1):
                    i += 1
                
                    
                #i = i + 1#??

                d = i - i_prev
                
                MyMIDI.addNote(track, channel, seq[i_prev], tick_count, d * one_tact / grain, volume)
                tick_count = tick_count + d * one_tact / grain 
                
                if (i == len(seq)):
                        break
                
                if (seq[i] == 0):
                    i_prev = i
                    if (i == len(seq) - 1):
                        break
                    while(seq[i] == 1 and i < len(seq) - 1):
                        i = i + 1
                        tick_count = tick_count + one_tact / grain 
                        

    
    print(track)

    with open("zzz3.mid", "wb") as output_file:
        MyMIDI.writeFile(output_file) 

In [23]:
p = take_one(model.predict(X_in))

In [62]:
with open("my_impr-0306.txt", "wb") as fp:   #Pickling
    pickle.dump(new_track, fp)



In [44]:
np.argmax(to_categorical(p, num_classes = 128))

43

In [21]:
encoded[4]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])

In [22]:
len(d)

524002

In [23]:
import tensorflow